### using XGbost model with diff paras 


In [1]:
import mlflow 

mlflow.set_registry_uri("http://127.0.0.1:5000")
mlflow.set_experiment("6.experimenting_XGbost")

c:\ProgramData\anaconda3\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/02/13 20:06:00 INFO mlflow.tracking.fluent: Experiment with name '6.experimenting_XGbost' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/1.DS%20projects/1-ML/youtube_comment_analysis/notebooks/mlruns/914715922977653067', creation_time=1770993360872, experiment_id='914715922977653067', last_update_time=1770993360872, lifecycle_stage='active', name='6.experimenting_XGbost', tags={}>

In [4]:
!pip install xgboost 
!pip install lightgbm

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
    --------------------------------------- 1.8/101.7 MB 6.3 MB/s eta 0:00:16
   - -------------------------------------- 3.9/101.7 MB 7.8 MB/s eta 0:00:13
   -- ------------------------------------- 5.5/101.7 MB 8.0 MB/s eta 0:00:13
   -- ------------------------------------- 7.3/101.7 MB 7.9 MB/s eta 0:00:12
   --- ------------------------------------ 8.9/101.7 MB 7.8 MB/s eta 0:00:12
   ---- ----------------------------------- 11.0/101.7 MB 8.2 MB/s eta 0:00:12
   ---- ----------------------------------- 12.6/101.7 MB 8.1 MB/s eta 0:00:11
   ----- ---------------------------------- 14.2/101.7 MB 8.1 MB/s eta 0:00:11
   ------ --------------------------------- 16.0/101.7 MB 8.2 MB/s eta 0:00:11
   ------- -------------------------------- 17.8/101.7 MB 8.4 MB/s eta 0:00:10
   ------- -------------------------------- 19.4/101.7 MB 8.1 MB/s eta 0:

In [5]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
     


df = pd.read_csv('processed.csv').dropna()
df.shape
     


(36662, 2)

In [ ]:


# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()

[I 2026-02-13 20:09:53,414] A new study created in memory with name: no-name-dc550a76-8299-4668-a3f8-b5ca2795c71f
[I 2026-02-13 20:12:43,618] Trial 0 finished with value: 0.5464339288149461 and parameters: {'n_estimators': 183, 'learning_rate': 0.000340481806461884, 'max_depth': 5}. Best is trial 0 with value: 0.5464339288149461.
[I 2026-02-13 20:17:28,615] Trial 1 finished with value: 0.7149870448656757 and parameters: {'n_estimators': 165, 'learning_rate': 0.041258514580192106, 'max_depth': 6}. Best is trial 1 with value: 0.7149870448656757.
[I 2026-02-13 20:24:55,126] Trial 2 finished with value: 0.7148506750306832 and parameters: {'n_estimators': 184, 'learning_rate': 0.02795792774052831, 'max_depth': 8}. Best is trial 1 with value: 0.7149870448656757.
[I 2026-02-13 20:35:13,890] Trial 3 finished with value: 0.804582026455748 and parameters: {'n_estimators': 257, 'learning_rate': 0.0723548536502451, 'max_depth': 10}. Best is trial 3 with value: 0.804582026455748.
[I 2026-02-13 20:5